The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [96]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [97]:
#Basic parameters
## Basic parameters
maxiter=1000;
max_time = 100;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [98]:
dataset = "rcv1_train.binary"   # mnist.scale  a9a  gisette_scale  rcv1_train.binary
prob = defineProb(dataset);
showDetails(dataset)

../data/rcv1_train.binary.jld


Dict{Symbol,Any} with 3 entries:
  :sparsity => 0.00156769
  :dims     => 47236
  :n        => 20242

In [99]:
norm(prob.A*prob.xsol - prob.b)

4.925422283141619e-8

In [100]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror) 
OUTPUTS = [];

In [101]:
# Testing Rademarcher Sketch
method_name = "rademacher";# ROS
options.sketchsize =10;
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-10
-------------------
It   | Error% | Residual |  Time   
-------------------
  3  | 92.63  |  87.63  | 123.7189 
127.944920 seconds (341.42 k allocations: 109.792 GB, 1.12% gc time)


In [102]:
# Testing Hadamard Sketch
# method_name = "Hadamard";# ROS
# options.sketchsize =2; # Int(floor(log2(prob.n)));
# options.maxiter=200;
# options.skip_error_calculation =25;
# @time  output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [103]:
# Testing countmin 
# options.sketchsize =floor((prob.n)^(1/2));
# options.skip_error_calculation =25;
# method_name = "countmin";# gradient descent
# @time output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [104]:
# Testing CG 
options.skip_error_calculation =25;
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 27  | 3.75  |  3.04  | 22.1130 
 50  | 1.26  |  0.83  | 44.2609 
 75  | 0.72  |  0.52  | 66.5034 
100  | 0.70  |  1.12  | 88.6146 
114  | 0.43  |  0.35  | 100.2478 
112.938200 seconds (18.56 k allocations: 341.559 MB, 0.11% gc time)


In [105]:
# Testing CD 
method_name = "CD";# gradient descent
options.sketchsize = floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-217
-------------------
It   | Error% | Residual |  Time   
-------------------
 82  | 45.11  |  26.77  | 4.7026 
 50  | 32.25  |  16.01  | 10.7601 
 75  | 25.52  |  12.01  | 15.4247 
100  | 20.72  |  9.25  | 19.9186 
125  | 16.88  |  7.38  | 25.0265 
150  | 14.55  |  6.36  | 29.8895 
175  | 12.74  |  5.73  | 34.9203 
200  | 10.98  |  5.06  | 39.8233 
225  | 9.67  |  4.33  | 44.2286 
250  | 8.74  |  4.02  | 48.9302 
275  | 8.01  |  3.75  | 53.9660 
300  | 7.44  |  3.52  | 58.9434 
325  | 6.48  |  3.32  | 63.6484 
350  | 6.61  |  3.11  | 68.3178 
375  | 5.72  |  2.90  | 72.9271 
400  | 5.41  |  2.74  | 78.5403 
425  | 5.06  |  2.66  | 82.9279 
450  | 4.75  |  2.48  | 87.5656 
475  | 4.45  |  2.31  | 92.3814 
500  | 4.25  |  2.25  | 97.3903 
515  | 4.13  |  2.20  | 100.0108 
141.168618 seconds (45.79 k allocations: 39.748 GB, 0.79% gc time)


In [108]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [109]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

rcv1_train.binary


In [16]:
# Loading saved problem data
probname = "w8a"#"gisette_scale-ridge";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];